# TP1 : Introduction aux LLMs

## 1. Installation des dépendances

In [ ]:
!pip install gradio openai 
!pip install transformers datasets torch datasets
!pip install scikit-learn pandas
!pip install fpdf fitz frontend pillow pdfplumber

## Instanciation d'un modèle LLM depuis Groq

In [ ]:
# Crée un client qui se connecte à l'API Groq via  API key
import os
import openai

# prérequis: créer un compte  et générer un token https://console.groq.com/keys

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass('Enter your groq api key')

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"]
)

In [ ]:
model_choosed="meta-llama/llama-4-scout-17b-16e-instruct"

## A. Simulation d'Entretien 

•	Génération de Questions : Utilisation d’un LLM léger pour générer dynamiquement des questions d’entretien en fonction du poste ou du profil souhaité. 
•	Interaction Conversationnelle : L’assistant peut jouer le rôle d’un recruteur en posant des questions et en enregistrant les réponses du candidat.

==> Génération de questions dynamiques à l’aide d’un LLM léger local (ou via transformers en fallback)

==> Interaction sous forme de Q/R avec transcription manuelle simulée (ou audio à ajouter ensuite)

==> Stockage des questions et réponses dans un log structuré (en vue d’une future analyse)

Prérequis :
    -transformers (pip install transformers)
    -(Optionnel) un LLM local comme Mistral ou TinyLLaMA via Ollama ou llama-cpp-python
    -whisper pour l’audio + transcription

### Imports

In [ ]:
import openai
import os
import json
from datetime import datetime

### Chat with model

In [ ]:
# Fonction générique pour interagir avec le modèle en streaming
def chat_with_model(prompt, display=True):
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    stream = client.chat.completions.create(
        model=model_choosed,  
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            response += content
            if display:
                print(content, end="")
    
    return response


### Fonction test si API / requête modele fonctionne

In [ ]:
prompt= "Peux-tu me raconter une blague ? "
reponse = chat_with_model(prompt)

### Simulation interactive de l’entretien

### Générateur de questions par rapport à un poste avec Interface gradio

In [ ]:
import gradio as gr

# Variables de session
session_log = {
    "profil": "",
    "questions_reponses": [],
    "previous_qs": [],
}

# Génération de question (même logique que plus tôt)
def generate_question_gradio(profil, model):
    global model_choosed
    model_choosed = model
    prompt = f"""Tu es un recruteur expérimenté. Pose une nouvelle question d'entretien à un candidat pour un poste de {profil}.
Évite de répéter les questions précédentes : {', '.join(session_log['previous_qs']) if session_log['previous_qs'] else 'aucune'}.
Donne uniquement la question, sans introduction, sans commentaire."""
    
    question = chat_with_model(prompt, display=False).strip()
    session_log["previous_qs"].append(question)
    return question

# Fonction principale d’interaction (question ↔ réponse)
def entretien_interactif(profil, model, reponse_utilisateur):
    if not session_log["profil"]:
        session_log["profil"] = profil

    if reponse_utilisateur and session_log["questions_reponses"]:
        # Sauvegarde réponse précédente
        session_log["questions_reponses"][-1]["reponse"] = reponse_utilisateur

    # Génère une nouvelle question
    question = generate_question_gradio(profil, model)
    session_log["questions_reponses"].append({
        "question": question,
        "reponse": ""
    })
    return question

# Fonction pour exporter les logs
def exporter_log():
    file_name = f"entretien_{session_log['profil'].replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(file_name, "w", encoding="utf-8") as f:
        json.dump(session_log, f, indent=2, ensure_ascii=False)
    return file_name


### Fonction analyse automatique des réponses (LLM)

In [ ]:
def analyser_entretien():
    if not session_log["questions_reponses"]:
        return "Aucune réponse à analyser."

    # Génère le contenu brut de l'entretien pour analyse
    entretien_text = "\n".join([
        f"Q: {pair['question']}\nR: {pair['reponse']}" 
        for pair in session_log["questions_reponses"]
    ])

    prompt = f"""
Tu es un expert RH. Analyse les réponses suivantes données lors d’un entretien pour un poste de {session_log['profil']} :

{entretien_text}

Donne un retour constructif :
- Points forts
- Points à améliorer
- Conseils pour le prochain entretien
Réponds de façon bienveillante et professionnelle.
"""
    feedback = chat_with_model(prompt, display=False)
    return feedback


### Fonction Évaluation des soft skills détectées

In [ ]:
def eval_soft_skills():
    entretien_text = "\n".join([
        f"Q: {pair['question']}\nR: {pair['reponse']}" 
        for pair in session_log["questions_reponses"]
    ])

    prompt = f"""
Lis cet échange d’entretien pour un poste de {session_log['profil']} :

{entretien_text}

Identifie les soft skills perçues dans les réponses du candidat (exemples : communication, leadership, adaptabilité, résolution de problèmes...).
Présente les résultats sous forme de liste avec un niveau estimé pour chaque compétence (faible, moyen, fort) et un commentaire.
"""
    soft_skills_feedback = chat_with_model(prompt, display=False)
    return soft_skills_feedback


### Génération de note globale de performance

In [ ]:
def generer_note_globale():
    entretien_text = "\n".join([
        f"Q: {pair['question']}\nR: {pair['reponse']}" 
        for pair in session_log["questions_reponses"]
    ])

    prompt = f"""
Voici un entretien pour un poste de {session_log['profil']} :

{entretien_text}

Attribue une note globale de performance (sur 10) au candidat, suivie d'un résumé de l'évaluation. Sois professionnel et bienveillant.
Exemple de sortie : "Note : 7/10 - Bon niveau technique, mais manque de précision sur certains points..."
"""
    result = chat_with_model(prompt, display=False)
    return result


### Génération + Téléchargement d’un PDF récapitulatif 

In [ ]:
from fpdf import FPDF
import tempfile

def generer_pdf_entretien():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, f"Compte-rendu d'entretien - {session_log['profil']}", ln=True)

    pdf.set_font("Arial", size=12)
    pdf.ln(5)

    for i, pair in enumerate(session_log["questions_reponses"], 1):
        pdf.multi_cell(0, 10, f"Q{i} : {pair['question']}")
        pdf.multi_cell(0, 10, f"R{i} : {pair['reponse']}")
        pdf.ln(2)

    # Feedback RH
    feedback = analyser_entretien()
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "🧠 Feedback RH", ln=True)
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, feedback)

    # Soft Skills
    softskills = eval_soft_skills()
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "📊 Soft Skills détectées", ln=True)
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, softskills)

    # Note globale
    note = generer_note_globale()
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "🎯 Note globale de performance", ln=True)
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, note)

    # Sauvegarde temporaire
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    pdf.output(temp_file.name)
    return temp_file.name


### Fonctionnalité : Upload de photo du candidat

Exemples de consignes personnalisables :
“Cette personne inspire-t-elle confiance pour un entretien ?”

“Cette tenue est-elle adaptée à un poste en entreprise ?”

“Fais une analyse non-verbale comme un coach en image.”

In [ ]:
import base64

def analyser_photo_candidat(image_path, commentaire="Analyse cette photo pour un entretien : posture, tenue, expression."):
    if not image_path:
        return "Aucune image chargée."

    # Encode l'image en base64
    with open(image_path, "rb") as f:
        image_bytes = f.read()
        image_b64 = base64.b64encode(image_bytes).decode('utf-8')
    
    image_data_url = f"data:image/jpeg;base64,{image_b64}"  # ou image/png selon le type

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": commentaire
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_data_url
                    }
                }
            ]
        }
    ]

    # Appel au modèle Vision (ex: llava-1.5-7b ou autre)
    stream = client.chat.completions.create(
        model=model_choosed,  # modèle vision compatible Groq
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            result += content
    return result


### [KO] Fonctionnalité : Upload de photo du candidat avec compression si > 30ko

In [ ]:
# TODO: KO compression
def analyser_photo_candidat_avec_preview(image_path, commentaire="Analyse cette photo pour un entretien : posture, tenue, expression."):
    if not image_path:
        return "Aucune image chargée.", None

    # Étape 1 : Compression + redimensionnement
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        img = img.resize((224, 224))
        buffered = io.BytesIO()
        img.save(buffered, format="JPEG", quality=30, optimize=True)
        image_bytes = buffered.getvalue()

        # Pour Gradio, on retourne un objet PIL directement ici
        img_preview = Image.open(io.BytesIO(image_bytes))

    # Vérif taille
    compressed_size_kb = len(image_bytes) / 1024
    if compressed_size_kb > 40:
        return f"❌ Image compressée trop lourde ({compressed_size_kb:.1f} Ko).", img_preview

    # Encodage base64 pour le LLM
    image_b64 = base64.b64encode(image_bytes).decode('utf-8')
    image_data_url = f"data:image/jpeg;base64,{image_b64}"

    # Construction du message pour le LLM
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": commentaire},
                {"type": "image_url", "image_url": {"url": image_data_url}}
            ]
        }
    ]

    # Appel API
    stream = client.chat.completions.create(
        model=model_choosed,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            result += content

    return result, img_preview


### Charger une fiche de poste (texte)

In [ ]:
def traiter_fiche_de_poste(texte_fiche, objectif="Génère 5 questions d'entretien adaptées à ce poste"):
    if not texte_fiche.strip():
        return "🛑 Merci de coller le contenu de la fiche de poste."

    # 🔒 Limiter la taille du texte pour éviter les erreurs du modèle
    max_chars = 4000  # Ajustable selon le modèle
    if len(texte_fiche) > max_chars:
        return f"⚠️ La fiche est trop longue ({len(texte_fiche)} caractères). Limite : {max_chars}."

    try:
        prompt = f"{objectif}\n\nVoici la fiche de poste :\n{texte_fiche}"

        messages = [
            {"role": "user", "content": prompt}
        ]

        stream = client.chat.completions.create(
            model=model_choosed,
            messages=messages,
            stream=True
        )

        result = ""
        for chunk in stream:
            content = chunk.choices[0].delta.content
            if content:
                result += content

        return result if result.strip() else "⚠️ Réponse vide du modèle."
    
    except Exception as e:
        return f"❌ Erreur : {str(e)}"


### [KO] Charger une fiche de poste (pdf)

In [ ]:
import fitz  # PyMuPDF pour lire les PDF

def lire_fiche_de_poste(fichier):
    if not fichier:
        return "Aucun fichier fourni."

    content = ""
    if fichier.name.endswith(".pdf"):
        doc = fitz.open(fichier.name)
        content = "\n".join([page.get_text() for page in doc])
    elif fichier.name.endswith(".txt"):
        with open(fichier.name, "r", encoding="utf-8") as f:
            content = f.read()
    else:
        return "Format non supporté. Utilisez un PDF ou un .txt."

    # On résume le contenu avec le LLM
    prompt = f"""
Voici une fiche de poste. Résume-la en identifiant :
- Le poste
- Les compétences clés attendues
- Les missions principales
- Le profil idéal

Fiche :
{content}
"""
    resume = chat_with_model(prompt, display=False)

    # On stocke dans la session pour l’utiliser dans la génération des questions
    session_log["profil"] = resume

    return resume


### [TODO] intégration fiche de poste dans la génération des questions

In [ ]:
# intégration dans la génération des questions
prompt = f"""
Tu joues le rôle d'un recruteur. Génère une question d'entretien adaptée à ce profil :

{session_log['profil']}

Pose une question à la fois. N’indique que la question sans commentaire ni réponse.
"""


### Interface utilisateur GRADIO

In [ ]:
with gr.Blocks(title="Assistant d'entretien IA") as demo:
    gr.Markdown("## 🤖 Assistant IA d'entraînement à l'entretien d'embauche")

    with gr.Row():
        poste_input = gr.Textbox(label="Profil / Poste visé", placeholder="ex: ingénieur IA, développeur web...")
        model_input = gr.Dropdown([model_choosed], 
                                  label="Modèle utilisé", value=model_choosed)

    # analyse photo
    gr.Markdown("## 📸 Analyse Visuelle du Candidat")
    photo_input = gr.Image(label="📷 Charge une photo", type="filepath")
    feedback_photo = gr.Textbox(label="🧠 Analyse de la photo", lines=5)
    bouton_photo = gr.Button("Analyser la photo")

    bouton_photo.click(fn=analyser_photo_candidat, inputs=[photo_input], outputs=[feedback_photo])

    # Upload fiche de post sous txt
    gr.Markdown("## 📋 Analyse de fiche de poste (coller le texte)")
    
    fiche_input = gr.Textbox(lines=10, placeholder="Collez ici le contenu de la fiche de poste", label="📝 Fiche de poste")
    objectif_input = gr.Textbox(value="Génère 5 questions d'entretien adaptées à ce poste", label="🎯 Objectif de génération")
    
    bouton = gr.Button("🔍 Analyser")
    output = gr.Textbox(label="📊 Résultat")

    bouton.click(
        fn=traiter_fiche_de_poste,
        inputs=[fiche_input, objectif_input],
        outputs=output
    )

    # Upload fiche de poste=
    comment='''
    fiche_input = gr.File(label="📄 Charger une fiche de poste (PDF ou .txt)")
    btn_fiche = gr.Button("📥 Lire la fiche de poste")
    fiche_resume = gr.Textbox(label="📝 Résumé de la fiche de poste", lines=6, interactive=False)
    btn_fiche.click(fn=lire_fiche_de_poste, inputs=[fiche_input], outputs=[fiche_resume])'''
   

    question_display = gr.Textbox(label="🧑‍💼 Question posée par le recruteur", interactive=False)
    reponse_input = gr.Textbox(label="🎤 Votre réponse", placeholder="Tapez votre réponse ici...")

    bouton_suivant = gr.Button("➡️ Question suivante")
    bouton_export = gr.Button("💾 Exporter les réponses")
    export_result = gr.Textbox(label="Fichier exporté", interactive=False)

    bouton_suivant.click(fn=entretien_interactif, 
                         inputs=[poste_input, model_input, reponse_input], 
                         outputs=[question_display])

    bouton_export.click(fn=exporter_log, inputs=[], outputs=[export_result])

    bouton_analyse = gr.Button("🧠 Analyse des réponses")
    feedback_output = gr.Textbox(label="🧠 Feedback RH personnalisé", lines=8, interactive=False)

    bouton_softskills = gr.Button("📊 Rapport Soft Skills")
    softskills_output = gr.Textbox(label="📊 Évaluation des soft skills", lines=8, interactive=False)

    bouton_analyse.click(fn=analyser_entretien, inputs=[], outputs=[feedback_output])
    bouton_softskills.click(fn=eval_soft_skills, inputs=[], outputs=[softskills_output])

    bouton_note = gr.Button("🎯 Générer note globale")
    note_output = gr.Textbox(label="🎯 Résultat de la performance", lines=3)

    bouton_pdf = gr.Button("📄 Télécharger le rapport PDF")
    pdf_output = gr.File(label="📥 Rapport PDF téléchargeable")

    bouton_note.click(fn=generer_note_globale, inputs=[], outputs=[note_output])
    # bouton_pdf.click(fn=generer_pdf_entretien, inputs=[], outputs=[pdf_output]);

### Lancement de l'interface

In [ ]:
demo.launch(share=True)  # Ajoute share=True pour un lien public sur smartphone